In [3]:
import numpy
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Flatten,Lambda,Conv2D,Dense,MaxPooling2D,Input,Dropout

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Correct rescale value
training_datagen = ImageDataGenerator(rescale=1./255)
test_datagen     = ImageDataGenerator(rescale=1./255)

train = r'C:\Users\jagad\OneDrive\Desktop\CNN from scratch\dataset\training_set'
test  = r'C:\Users\jagad\OneDrive\Desktop\CNN from scratch\dataset\test_set'

training_set = training_datagen.flow_from_directory(
    train,
    batch_size=32,
    target_size=(224, 224),
    class_mode='binary'  
)
test_set = test_datagen.flow_from_directory(
    test,
    batch_size=32,
    target_size=(224, 224),
    class_mode='binary'   
)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


# creating the CNN model

In [47]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import keras_tuner as kt

def create_model(hp):
    model = Sequential()

    model.add(Conv2D(
        filters=hp.Int('conv1', min_value=8, max_value=64, step=8),
        kernel_size=(3, 3),
        activation='relu',
        input_shape=(224, 224, 3),
        name='cnn_firstlayer'
    ))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(
        filters=hp.Int('conv2', min_value=16, max_value=128, step=16),
        kernel_size=(3, 3),
        activation='relu',
        name='cnn_secondlayer'
    ))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())

    model.add(Dropout(
        hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1),
        name='Drop_out1'
    ))

    model.add(Dense(
        units=hp.Int("dense1", min_value=32, max_value=256, step=32),
        activation='relu',
        name='Dense_layer'
    ))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model


In [48]:
tuner_search = kt.RandomSearch(
    create_model,
    objective='val_accuracy',
    max_trials=7,
    executions_per_trial=2,
    directory='tunersearch',
    project_name='dog_cat_recognition'
)


In [51]:
tuner_search.search(training_set,epochs=5,validation_data=test_set,)

Trial 7 Complete [00h 12m 10s]
val_accuracy: 0.7002499997615814

Best val_accuracy So Far: 0.7317500114440918
Total elapsed time: 02h 27m 30s


In [60]:
best_model = tuner_search.get_best_models(1)[0]




c:\Users\jagad\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\jagad\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [61]:
best_model.save("cat_dog_model.h5")